## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [2]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [6]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        self.W1 = tf.Variable(tf.random.normal([784, 256], stddev=0.1))
        self.b1 = tf.Variable(tf.zeros([256]))
        self.W2 = tf.Variable(tf.random.normal([256, 128], stddev=0.1))
        self.b2 = tf.Variable(tf.zeros([128]))
        self.W3 = tf.Variable(tf.random.normal([128, 10], stddev=0.1))
        self.b3 = tf.Variable(tf.zeros([10]))

        ####################
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        x = tf.reshape(x, [-1, 784])
        h1 = x @ self.W1 + self.b1
        h1 = tf.nn.relu(h1)
        h2 = h1 @ self.W2 + self.b2
        h2 = tf.nn.relu(h2)
        logits = h2 @ self.W3 + self.b3
        
        ####################
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [7]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [9]:
train_data, test_data = mnist_dataset()
for epoch in range(500):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.0264313 ; accuracy 0.32096666
epoch 1 : loss 2.0188155 ; accuracy 0.32508335
epoch 2 : loss 2.0112865 ; accuracy 0.32985
epoch 3 : loss 2.0038378 ; accuracy 0.33523333
epoch 4 : loss 1.9964659 ; accuracy 0.34055
epoch 5 : loss 1.9891695 ; accuracy 0.34588334
epoch 6 : loss 1.9819443 ; accuracy 0.35031667
epoch 7 : loss 1.9747872 ; accuracy 0.35535
epoch 8 : loss 1.9676971 ; accuracy 0.36001667
epoch 9 : loss 1.9606704 ; accuracy 0.36433333
epoch 10 : loss 1.9537027 ; accuracy 0.3691
epoch 11 : loss 1.9467925 ; accuracy 0.37331668
epoch 12 : loss 1.9399389 ; accuracy 0.3783
epoch 13 : loss 1.9331405 ; accuracy 0.38308334
epoch 14 : loss 1.926395 ; accuracy 0.3877
epoch 15 : loss 1.9197 ; accuracy 0.39211667
epoch 16 : loss 1.9130522 ; accuracy 0.3961
epoch 17 : loss 1.906451 ; accuracy 0.4004
epoch 18 : loss 1.8998951 ; accuracy 0.4048
epoch 19 : loss 1.8933825 ; accuracy 0.40875
epoch 20 : loss 1.8869117 ; accuracy 0.41271666
epoch 21 : loss 1.8804817 ; accuracy 0.4169